<a href="https://colab.research.google.com/github/mdias23i/DE-DataProcessing/blob/main/spark/examples/06-write_partitioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Write
- .write
- .format (parquet, csv, json)
- options
- spark.sql.sources.partitionOverwriteMode dynamic

# Write Mode
- overwrite - The overwrite mode is used to overwrite the existing file, alternatively, you can use SaveMode.Overwrite
- append - To add the data to the existing file, alternatively, you can use SaveMode.Append
- ignore - Ignores write operation when the file already exists, alternatively, you can use SaveMode.Ignore.
- errorifexists or error - This is a default option when the file already exists, it returns an error, alternatively, you can use SaveMode.ErrorIfExists.

# Partitioning
Process to organize the data into multiple chunks based on some criteria.
Partitions are organized in sub-folders.
Partitioning improves performance in Spark.

# Setting up PySpark

In [ ]:
%pip install pyspark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').getOrCreate()

# Preparing data

In [2]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.4 MB/s eta 0:00:00


In [3]:
from faker import Faker
from datetime import datetime

fake = Faker()

users = []
for _ in range(50):
    user = {
        'date': fake.date_time_between_dates(datetime(2024, 5, 1), datetime(2024, 5, 5)),
        'name': fake.name(),
        'address': fake.address(),
        'email': fake.email(),
        'dob': fake.date_of_birth(),
        'phone': fake.phone_number()
    }
    users.append(user)

df = spark.createDataFrame(users)

df.show(10, False)


+--------------------------------------------------------+--------------------------+----------+-----------------------+--------------------+--------------------+
|address                                                 |date                      |dob       |email                  |name                |phone               |
+--------------------------------------------------------+--------------------------+----------+-----------------------+--------------------+--------------------+
|24647 Carmen Mall\nPort Donnaview, ME 69957             |2024-05-01 18:28:09.301006|1945-07-24|graybrian@example.org  |Jason Dunlap        |(668)492-0513       |
|06846 Mills Dale Suite 012\nBrianside, MO 38344         |2024-05-01 14:49:29.585798|1956-06-16|jason53@example.org    |Andrew James        |001-548-973-2383x852|
|107 Robles Wells\nPort Michelle, PW 58336               |2024-05-03 19:00:37.186797|1946-02-21|hlittle@example.com    |Lisa Roberts        |(916)863-3834x0794  |
|88632 Rodney Plain\nL

# Writing as PARQUET



In [4]:
# Writing as PARQUET with no partitions

path = "/content/write_partitioning/parquet_no_partitions"

df.write.mode("overwrite").format("parquet").save(path)

!ls /content/write_partitioning/parquet_no_partitions

spark.read.format("parquet").load(path).count()

part-00000-3f09c745-5727-4b55-97b9-f219dbf703e3-c000.snappy.parquet  _SUCCESS


50

In [5]:
# Writing as PARQUET with partitions
from pyspark.sql.functions import *

path = "/content/write_partitioning/parquet_with_partitions"

# Creating partition column
df = df.withColumn("date_part", date_format(col("date"), "yyyyMMdd"))

spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic") # enable dynamic partition overwrite - only overwrites partitions that are coming in the dataframe

(df#.where("date_part = '20240503'")
 .write
 .mode("overwrite")                                               # overwrites the entire path with the new data
 .partitionBy("date_part")                                        # partition the data by column - creates sub-folders for each partition
 .format("parquet")                                               # format of output
 .save(path))                                                     # path

!ls /content/write_partitioning/parquet_with_partitions

spark.read.format("parquet").load(path).count()

'date_part=20240501'  'date_part=20240502'  'date_part=20240503'  'date_part=20240504'


50

In [6]:
# Checking single partition
spark.read.parquet("/content/write_partitioning/parquet_with_partitions/date_part=20240502").show()

+--------------------+--------------------+----------+--------------------+---------------+--------------------+
|             address|                date|       dob|               email|           name|               phone|
+--------------------+--------------------+----------+--------------------+---------------+--------------------+
|51846 Scott Mount...|2024-05-02 17:01:...|1992-12-27|rodney65@example.net|    Russell Cox|          3144378208|
|16120 James Lakes...|2024-05-02 11:35:...|1936-08-19| jayford@example.com|  Jamie Ramirez|     +1-899-543-8290|
|8263 Jermaine Wal...|2024-05-02 19:02:...|1957-09-17| zsnyder@example.net| Erin Castaneda|001-512-806-6732x...|
|224 Kemp Row Apt....|2024-05-02 00:28:...|1989-04-12|  psmith@example.net|Willie Mcdonald|  257-557-4643x12304|
|1285 Eric Gateway...|2024-05-02 20:12:...|1941-04-28|kimberlyirwin@exa...|  Ashley Guzman|001-926-842-7730x...|
|79212 Nicole Fiel...|2024-05-02 00:04:...|1915-02-10|mannjaime@example...|Elizabeth Logan|  (49

# Writing as CSV

https://spark.apache.org/docs/3.5.1/sql-data-sources-csv.html

In [7]:
df.count()

50

In [9]:
path = "/content/write_partitioning/csv_no_partitioning/"

# write as csv
(df
  .write
  .format("csv")
  .mode("overwrite")
  .option("delimiter", "|")
  .option("header", True)
  .save(path))

# listing files in the folder
!ls /content/write_partitioning/csv_no_partitioning/

# read as csv
(spark
  .read
  .options(sep="|", multiLine=True, header=True)
  .csv(path)
  .count())

part-00000-714d7354-65e9-4aa9-948b-9f76747e2cee-c000.csv  _SUCCESS


50

# Writing as JSON

https://spark.apache.org/docs/3.5.1/sql-data-sources-json.html

In [11]:
path = "/content/write_partitioning/json_no_partitioning/"

# write as json
(df
.write
.mode("overwrite")
.format("json")
.save(path))

# listing files in the folder
!ls /content/write_partitioning/json_no_partitioning/

# read as json
(spark
  .read
  .json(path)
  .count())

part-00000-b7d5210d-787a-4e93-b06d-81de03f3da7e-c000.json  _SUCCESS


50

In [12]:
# reading json as text
spark.read.text(path).show(10, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                               |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"address":"24647 Carmen Mall\nPort Donnaview, ME 69957","date":"2024-05-01T18:28:09.301Z","dob":"1945-07-24","email":"graybrian@example.org","name":"Jason Dunlap","phone":"(668)492-0513","date_part":"20240501"}                 |
|{"address":"06846 Mills Dale Suite 012\nBrianside, MO 38344","date":"2024-0

In [13]:
# reading json as text
spark.read.json(path).show(10, False)

+--------------------------------------------------------+------------------------+---------+----------+-----------------------+--------------------+--------------------+
|address                                                 |date                    |date_part|dob       |email                  |name                |phone               |
+--------------------------------------------------------+------------------------+---------+----------+-----------------------+--------------------+--------------------+
|24647 Carmen Mall\nPort Donnaview, ME 69957             |2024-05-01T18:28:09.301Z|20240501 |1945-07-24|graybrian@example.org  |Jason Dunlap        |(668)492-0513       |
|06846 Mills Dale Suite 012\nBrianside, MO 38344         |2024-05-01T14:49:29.585Z|20240501 |1956-06-16|jason53@example.org    |Andrew James        |001-548-973-2383x852|
|107 Robles Wells\nPort Michelle, PW 58336               |2024-05-03T19:00:37.186Z|20240503 |1946-02-21|hlittle@example.com    |Lisa Roberts     

In [15]:
# partition json data + saveAsTable

# Creating partition column
df = df.withColumn("date_part", date_format(col("date"), "yyyyMMdd"))

# write as json
(df.write
  .partitionBy("date_part")
  .mode("overwrite")
  .format("json")
  .saveAsTable("tbl_json_part"))

# read as json
print(spark.table("tbl_json_part").count())

# read as json
spark.sql("show partitions tbl_json_part").show()

50
+------------------+
|         partition|
+------------------+
|date_part=20240501|
|date_part=20240502|
|date_part=20240503|
|date_part=20240504|
+------------------+



# Append Mode

In [17]:
# Writing as PARQUET with APPEND

path = "/content/write_partitioning/parquet_append"

df.write.mode("append").format("parquet").save(path)

!ls /content/write_partitioning/parquet_append

spark.read.format("parquet").load(path).count()

part-00000-297caa6f-52e2-41ac-bb50-9598d31798e8-c000.snappy.parquet  _SUCCESS
part-00000-30a73df1-3e66-40c1-a756-2814d529955c-c000.snappy.parquet


100